# Key Steps Flow (Selectable Cases)

选择少量题目（1条、10条等）打印关键步骤：
- 仅打印 QUESTION / GOLD
- 打印 LLM 阶段A Draft 与 阶段B Reflection 的原始输出
- 打印最终纠偏后的结果
- 不打印因果三元组/构图细节（内部使用静默反思版）


In [1]:
import os, json, random
import pandas as pd
from datetime import datetime

# 仅打印关键流程的工具（调用静默反思版）
from llm_predictors_flow import init_defaults, run_cases

# 默认模型与轻量构图配置
MODEL = 'gemma2:27b'
MAX_EXPANSION_DEPTH = 2
MAX_NEIGHBORS_PER_SEED = 5
MAX_RELATIONS_PER_ENTITY = 5

# 初始化（如需自定义，可直接修改上方参数）
PARSER, BUILDER, MODEL = init_defaults(
    model=MODEL,
    max_depth=MAX_EXPANSION_DEPTH,
    max_neighbors=MAX_NEIGHBORS_PER_SEED,
    max_relations=MAX_RELATIONS_PER_ENTITY,
)
print('Key steps flow ready.')


Key steps flow ready.


In [5]:
# 选择模式与样本数
SAMPLE_MODE = 'custom'  # 'custom' or 'random'
NUM_SAMPLES = 10       # 例如 1 或 10
SEED = 42
WIQA_PATH = 'wiqa_train_data.json'  # 本地 NDJSON 文件

# 自定义用例（可留空；字段：question, ground_truth 可选）
custom_cases = [
    {
        'question': 'suppose the seedling is not eaten happens, how will it affect LESS trees?',
        'ground_truth': 'less'
    }
]

def load_wiqa_local(path=WIQA_PATH, limit=10, seed=42):
    items = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                obj = json.loads(line)
            except Exception:
                continue
            q = obj.get('question_stem') or obj.get('question') or ''
            lbl = (obj.get('answer_label') or obj.get('label') or '').strip().lower()
            if q:
                items.append({'question': q, 'ground_truth': lbl or None})
    random.Random(seed).shuffle(items)
    return items[:limit]

print('Config set. Mode:', SAMPLE_MODE, '| N =', NUM_SAMPLES)


Config set. Mode: custom | N = 10


In [6]:
cases

[{'question': 'suppose the seedling is not eaten happens, how will it affect LESS trees?',
  'ground_truth': 'less'}]

In [8]:
# 执行并打印关键步骤
# if SAMPLE_MODE == 'custom' and len(custom_cases) > 0:
#     cases = custom_cases
# else:
#     cases = load_wiqa_local(limit=NUM_SAMPLES, seed=SEED)
cases = load_wiqa_local(limit=NUM_SAMPLES, seed=SEED)

results = run_cases(cases, parser=PARSER, builder=BUILDER, model=MODEL)
df = pd.DataFrame(results)
print('Summary table:')
display(df)



QUESTION:
suppose getting a storm over the coast from the ocean happens, how will it affect MORE erosion by the ocean.
GOLD: more
--------------------------------------------------------------------------------
[Meta-Informed] Stage A - Draft (LLM raw):
```json
{
  "entity_effect": "increase",
  "label_guess": "more",
  "rationale": "Storms generate large waves and strong currents. These powerful forces batter the coastline, dislodging sediment and rock, leading to increased erosion."
}
```
[Meta-Informed] Stage B - Reflection (LLM raw):
```json
{
  "computed_label": "more",
  "final_label": "more",
  "correction_reason": "",
  "confidence": 0.95
}
```
[Meta-Informed] Final: more | corrected=False | source=none
--------------------------------------------------------------------------------
[Combined-Context] Stage A - Draft (LLM raw):
```json
{
  "entity_effect": "increase",
  "label_guess": "more",
  "rationale": "Storms generate strong waves and high tides. These powerful forces di

,question,gold,meta_final,meta_corrected,comb_final,comb_corrected
0,suppose getting a storm over the coast from th...,more,more,False,more,False
1,"suppose use less batter per pancake happens, h...",no_effect,more,False,no_effect,True
2,"suppose Having a stomach bug happens, how will...",no_effect,no_effect,True,no_effect,True
3,"suppose more caterpillar eats happens, how wil...",no_effect,more,False,more,False
4,suppose The plant is given a new contract for ...,less,no_effect,True,less,True
5,suppose there is oxidization happening on iron...,less,less,False,more,True
6,"suppose people recycle less happens, how will ...",more,no_effect,True,more,False
7,suppose more minerals dissolved in the water h...,no_effect,more,False,no_effect,False
8,suppose a person may begin to hyperventilate h...,no_effect,no_effect,False,less,False
9,"suppose more fossil fuels burned happens, how ...",no_effect,more,False,more,False
